# Keras

In [14]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

from keras.layers import Embedding, Input, Dense
import keras
import keras.backend as K

import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Feature Engineering

In [1]:
### Thanks to Shinji Suzuki

# OS
import glob, re
from datetime import datetime
import pickle

# data science tool
import numpy as np
import pandas as pd
import datetime as dt


# machine learning
from sklearn import *
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb

# データの読み込み
# 事前にcalendar_dateをvisit_dataに変更しています。airとhpgで同じことですが、別名で使用されているようです。
data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
}

# それぞれのデータをマージするために、まずは、relation用のものをマージします
data['hr'] = pd.merge(data['hr'], data['id'], how = 'inner', on = ['hpg_store_id'])

for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r:(r['visit_datetime']- r['reserve_datetime']).days, axis = 1)
    tmp1 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how = 'inner', on = ['air_store_id', 'visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()

stores = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'dow':[i]*len(unique_stores)}) for i in range(7)], axis =0, ignore_index = True).reset_index(drop = True) 

#曜日だけでなく、月も追加
stores_m = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'month':[i]*len(unique_stores)}) for i in range(1,13)], axis =0, ignore_index = True).reset_index(drop = True)
stores = pd.merge(stores_m, stores,on=('air_store_id'), how='left')

tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].min().rename(columns = {'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].median().rename(columns = {'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].max().rename(columns = {'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].count().rename(columns = {'visitors':'count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])

#曜日だけでなく、ID×月も追加
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].min().rename(columns = {'visitors':'m_min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'m_mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].median().rename(columns = {'visitors':'m_median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].max().rename(columns = {'visitors':'m_max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].count().rename(columns = {'visitors':'m_count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])

stores = pd.merge(stores, data['as'], how= "left", on = ['air_store_id'])

stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/', ' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-', ' ')))

lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name' + str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
    stores['air_area_name' + str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

#土日フラグ(day_of_week_1)と、休日前(holi_2)フラグを追加
data['hol']['day_of_week_1']= data['hol']['day_of_week'].replace(['Saturday', 'Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'],['1', '1','0','0','0','0','0']).astype('int')
data['hol']['holi_2'] = data['hol'][['holiday_flg', 'day_of_week_1']].sum(axis = 1)
data['hol']['holi_2'] = data['hol']['holi_2'].apply( lambda x: 0 if x < 1 else 1 )
data['hol']['holi_2'] = data['hol']['holi_2'].shift(-1)
data['hol']['holi_2'] = data['hol']['holi_2'].fillna(1)
data['hol']['holi_2'] = data['hol']['holi_2'].astype('int')

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how ='left', on = ['visit_date'])
test = pd.merge(data['tes'], data['hol'], how ='left', on = ['visit_date'])

#曜日と月でmerge
train = pd.merge(train, stores, how ='left', on = ['air_store_id', 'dow','month'])
test = pd.merge(test, stores, how ='left', on = ['air_store_id', 'dow','month'])

#ID×休日前でのvisitorsの平均、中央値等を追加
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].min().rename(columns = {'visitors':'h_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'h_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].median().rename(columns = {'visitors':'h_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].max().rename(columns = {'visitors':'h_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].count().rename(columns = {'visitors':'h_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1) 

train['total_reserve_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserve_mean'] = (train['rv2_x'] + train['rv2_y'])/2
train['total_reserve_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y'])/2

test['total_reserve_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserve_mean'] = (test['rv2_x'] + test['rv2_y'])/2
test['total_reserve_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y'])/2

train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2']= lbl.fit_transform(test['air_store_id'])

train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date', 'visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat([train, test]) 

#指数移動平均の追加。これはなくても良いかも
#https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/46179#266344
#def calc_shifted_ewm(series, alpha, adjust=True):
#    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()

#train['ewm'] = train.groupby(['air_store_id', 'dow']).apply(lambda g: calc_shifted_ewm(g['visitors'], 0.1)).sort_index(level=['air_store_id']).values

#以下、気象データの追加
df_air_store_weather_station = pd.read_csv('../../../mltestdata/05_recruit/air_store_info_with_nearest_active_station.csv')

cols = ['air_store_id', 'station_id', 'station_latitude', 'station_longitude', 'station_vincenty', 'station_great_circle']
all_data = pd.merge(all_data, df_air_store_weather_station[cols], on='air_store_id', how='left')

combine = all_data
filenames = []
df_weather = None
for station_id in combine['station_id'].unique():
    fn = f"../../../mltestdata/05_recruit/1-1-16_5-31-17_Weather/{station_id}.csv"
    if not fn in filenames:
        df = pd.read_csv(fn)
        df['station_id'] = station_id
        if df_weather is None:
            df_weather = df
        else:
            df_weather = pd.concat([df_weather, df])
        del df

        filenames.append(fn)
    else:
        continue

#欠損値を平均で穴埋め（median, ffillで試すも特に差は出なかった）
df_weather = df_weather.fillna(df_weather.mean())

df_weather = df_weather.rename(columns={'calendar_date': 'visit_date'})

df_weather['visit_date'] = pd.to_datetime(df_weather['visit_date'])
df_weather['visit_date'] = df_weather['visit_date'].dt.date

#なんとなく対数化
df_weather['precipitation'] = np.log1p(df_weather['precipitation'])

#使いそうなデータだけ結合（その他の気象データは試していません。特に意味はなし）
cols = ['station_id', 
    'visit_date', 
    'precipitation', 
    'hours_sunlight',
    'avg_temperature',
    'high_temperature',
    'low_temperature']

combine = pd.merge(combine, df_weather[cols], on=['station_id', 'visit_date'], how='left')

#降水量をカテゴリ化
def simplify_pre(df):
    df.precipitation = df.precipitation.fillna(0)
    bins = ( -1, 0.01, 2,  5)
    group_names = ['1', '2', '3']
    categories = pd.cut(df.precipitation, bins, labels=group_names)
    df.precipitation = categories
    return df

combine = simplify_pre(combine) 
all_data = combine 

#不要そうなデータを削除
drop_col =['station_id', 'station_latitude','station_longitude','station_vincenty', 'station_great_circle','hours_sunlight','high_temperature','low_temperature']
all_data = all_data.drop(drop_col, axis = 1)

train = all_data[:ntrain]
test = all_data[ntrain:]

#ID×降水量で平均、中央値等を追加
tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].min().rename(columns = {'visitors':'p_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'p_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].median().rename(columns = {'visitors':'p_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation',])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].max().rename(columns = {'visitors':'p_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].count().rename(columns = {'visitors':'p_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

#countをLabel Encoder化。なんとなく試してみたら結果が良かった。
lbl = preprocessing.LabelEncoder()
train['count_visitors'] = lbl.fit_transform(train['count_visitors']) 
test['count_visitors']= lbl.fit_transform(test['count_visitors'])
train['m_count_visitors'] = lbl.fit_transform(train['m_count_visitors'])
test['m_count_visitors']= lbl.fit_transform(test['m_count_visitors'])
train['h_count_visitors'] = lbl.fit_transform(train['h_count_visitors'])
test['h_count_visitors']= lbl.fit_transform(test['h_count_visitors'])
#train['h_count_reserves'] = lbl.fit_transform(train['h_count_visitors'])
#test['h_count_reserves']= lbl.fit_transform(test['h_count_visitors'])
train['p_count_visitors'] = lbl.fit_transform(train['p_count_visitors'])
test['p_count_visitors']= lbl.fit_transform(test['p_count_visitors'])

# GW flag
combine = [train, test]
gw_list = ['2016-04-29','2016-04-30','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05']
post_gw_list=['2016-05-06']
train['gw_flg'] = 0
train['post_gw_flg'] = 0
test['gw_flg'] = 0
test['post_gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
    
for index, gw_date in enumerate(post_gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_post_gw_list[index][col_i]=int(temp_figure)

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
        
for dataset in combine:
    for index in range(len(update_post_gw_list)):
        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1     

/Users/suzukiry/lab/March/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/suzukiry/lab/March/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/suzukiry/lab/March/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be remove

In [2]:
train['precipitation'] = train['precipitation'].values.astype(float)
test['precipitation'] = test['precipitation'].values.astype(float)

In [3]:
# Handle NaN
train = train.fillna(-1)
test = test.fillna(-1)

In [46]:
drop_cols=['visitors','air_store_id','visit_date','id']
#drop_cols=['air_store_id','visit_date','id']
y_train=train['visitors']
x_train=train.drop(drop_cols, axis=1)

x_test=test.copy()
x_test=x_test.drop(drop_cols, axis=1)

In [47]:
y = train.visitors
train_input = train.copy()
test_input = test.copy()

drop_cols=['visitors','air_store_id','visit_date','id']
train_input=train_input.drop(drop_cols, axis=1)
test_input=test_input.drop(drop_cols, axis=1)

# Define utility tool

In [48]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

# Data preparation

In [33]:
pd.set_option("display.max_rows", 101)

In [34]:
train.shape

(252108, 75)

In [35]:
train.precipitation.dtypes

dtype('float64')

In [36]:
#value_col: taken as float input(which are normalized)
#nn_col - value_col: taken as categorical inputs(embedding layers used)

value_col_list = []
nn_col_list = []

for c, dtype in zip(train.columns, train.dtypes):	
    if dtype == np.object:		
        value_col_list.append(c)

    elif dtype == np.int64 :		
        value_col_list.append(c)
        
    elif dtype == np.float64:
        nn_col_list.append(c)

    else:
        print(c)

print("value_col(n={}): {} ".format(len(value_col_list),value_col_list))
print("\nnn_col(n={}): {} ".format(len(nn_col_list),nn_col_list))

value_col(n=21): ['air_store_id', 'air_store_id2', 'count_visitors', 'date_int', 'day_of_week', 'day_of_week_1', 'dow', 'h_count_visitors', 'h_max_visitors', 'h_min_visitors', 'holi_2', 'holiday_flg', 'id', 'm_count_visitors', 'month', 'visit_date', 'visitors', 'year', 'p_count_visitors', 'gw_flg', 'post_gw_flg'] 

nn_col(n=54): ['air_area_name', 'air_area_name0', 'air_area_name1', 'air_area_name2', 'air_area_name3', 'air_area_name4', 'air_area_name5', 'air_area_name6', 'air_area_name7', 'air_area_name8', 'air_area_name9', 'air_genre_name', 'air_genre_name0', 'air_genre_name1', 'air_genre_name2', 'air_genre_name3', 'air_genre_name4', 'air_genre_name5', 'air_genre_name6', 'air_genre_name7', 'air_genre_name8', 'air_genre_name9', 'h_mean_visitors', 'h_median_visitors', 'latitude', 'lon_plus_lat', 'longitude', 'm_max_visitors', 'm_mean_visitors', 'm_median_visitors', 'm_min_visitors', 'max_visitors', 'mean_visitors', 'median_visitors', 'min_visitors', 'rs1_x', 'rs1_y', 'rs2_x', 'rs2_y', 'r

In [37]:
value_col = ['holiday_flg',
             'min_visitors',
             'mean_visitors',
             'median_visitors',
             'max_visitors',
             #'count_observations',
             'rs1_x',
             'rv1_x',
             'rs2_x',
             'rv2_x',
             'rs1_y',
             'rv1_y',
             'rs2_y',
             'rv2_y',
             'total_reserve_sum',
             'total_reserve_mean',
             'total_reserve_dt_diff_mean',
             'date_int',
             'var_max_lat',
             'var_max_long',
             'lon_plus_lat',
             'h_max_visitors',
             'h_mean_visitors',
             'h_median_visitors',
             'h_min_visitors',
             'm_max_visitors', 
             'm_mean_visitors', 
             'm_median_visitors', 
             'm_min_visitors',
             'p_min_visitors', 
             'p_mean_visitors', 
             'p_median_visitors', 
             'p_max_visitors',
             'latitude',
             'longitude',  
            ]

nn_col = value_col + ['dow', 
                      'day_of_week', 
                      'day_of_week_1', 
                      'gw_flg',
                      'post_gw_flg',
                      'holi_2',             
                      'h_count_visitors',
                      'm_count_visitors',
                      'p_count_visitors',
                      'precipitation',
                      'year', 
                      'month', 
                      'air_store_id2', 
                      'air_area_name', 
                      'air_genre_name',
                      'air_area_name0', 
                      'air_area_name1', 
                      'air_area_name2', 
                      'air_area_name3', 
                      'air_area_name4',
                      'air_area_name5', 
                      'air_area_name6', 
                      'air_area_name7',
                      'air_area_name8', 
                      'air_area_name9', 
                      'air_genre_name0', 
                      'air_genre_name1',
                      'air_genre_name2', 
                      'air_genre_name3', 
                      'air_genre_name4',
                      'air_genre_name5',
                      'air_genre_name6', 
                      'air_genre_name7', 
                      'air_genre_name8',
                      'air_genre_name9'
                     ]

In [42]:
X = train.copy()
X_test = test[nn_col].copy()

value_scaler = preprocessing.MinMaxScaler()
for vcol in value_col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

X_train = list(X[nn_col].T.as_matrix())
Y_train = np.log1p(X['visitors']).values
nn_train = [X_train, Y_train]
nn_test = [list(X_test[nn_col].T.as_matrix())]
print("Train and test data prepared")

Train and test data prepared


In [ ]:
def get_nn_complete_model(train, hidden1_neurons=35, hidden2_neurons=15):
    """
    Input:
        train:           train dataframe(used to define the input size of the embedding layer)
        hidden1_neurons: number of neurons in the first hidden layer
        hidden2_neurons: number of neurons in the first hidden layer
    Output:
        return 'keras neural network model'
    """
    K.clear_session()

    air_store_id = Input(shape=(1,), dtype='int32', name='air_store_id')
    air_store_id_emb = Embedding(len(train['air_store_id2'].unique()) + 1, 15, input_shape=(1,),
                                 name='air_store_id_emb')(air_store_id)
    air_store_id_emb = keras.layers.Flatten(name='air_store_id_emb_flatten')(air_store_id_emb)

    dow = Input(shape=(1,), dtype='int32', name='dow')
    dow_emb = Embedding(8, 3, input_shape=(1,), name='dow_emb')(dow)
    dow_emb = keras.layers.Flatten(name='dow_emb_flatten')(dow_emb)

    month = Input(shape=(1,), dtype='int32', name='month')
    month_emb = Embedding(13, 3, input_shape=(1,), name='month_emb')(month)
    month_emb = keras.layers.Flatten(name='month_emb_flatten')(month_emb)

    air_area_name, air_genre_name = [], []
    air_area_name_emb, air_genre_name_emb = [], []
    for i in range(7):
        area_name_col = 'air_area_name' + str(i)
        air_area_name.append(Input(shape=(1,), dtype='int32', name=area_name_col))
        tmp = Embedding(len(train[area_name_col].unique()), 3, input_shape=(1,),
                        name=area_name_col + '_emb')(air_area_name[-1])
        tmp = keras.layers.Flatten(name=area_name_col + '_emb_flatten')(tmp)
        air_area_name_emb.append(tmp)

        if i > 4:
            continue
        area_genre_col = 'air_genre_name' + str(i)
        air_genre_name.append(Input(shape=(1,), dtype='int32', name=area_genre_col))
        tmp = Embedding(len(train[area_genre_col].unique()), 3, input_shape=(1,),
                        name=area_genre_col + '_emb')(air_genre_name[-1])
        tmp = keras.layers.Flatten(name=area_genre_col + '_emb_flatten')(tmp)
        air_genre_name_emb.append(tmp)

    air_genre_name_emb = keras.layers.concatenate(air_genre_name_emb)
    air_genre_name_emb = Dense(4, activation='sigmoid', name='final_air_genre_emb')(air_genre_name_emb)

    air_area_name_emb = keras.layers.concatenate(air_area_name_emb)
    air_area_name_emb = Dense(4, activation='sigmoid', name='final_air_area_emb')(air_area_name_emb)
    
    air_area_code = Input(shape=(1,), dtype='int32', name='air_area_code')
    air_area_code_emb = Embedding(len(train['air_area_name'].unique()), 8, input_shape=(1,), name='air_area_code_emb')(air_area_code)
    air_area_code_emb = keras.layers.Flatten(name='air_area_code_emb_flatten')(air_area_code_emb)
    
    air_genre_code = Input(shape=(1,), dtype='int32', name='air_genre_code')
    air_genre_code_emb = Embedding(len(train['air_genre_name'].unique()), 5, input_shape=(1,),
                                   name='air_genre_code_emb')(air_genre_code)
    air_genre_code_emb = keras.layers.Flatten(name='air_genre_code_emb_flatten')(air_genre_code_emb)

    
    holiday_flg = Input(shape=(1,), dtype='float32', name='holiday_flg')
    year = Input(shape=(1,), dtype='float32', name='year')
    min_visitors = Input(shape=(1,), dtype='float32', name='min_visitors')
    mean_visitors = Input(shape=(1,), dtype='float32', name='mean_visitors')
    median_visitors = Input(shape=(1,), dtype='float32', name='median_visitors')
    max_visitors = Input(shape=(1,), dtype='float32', name='max_visitors')
    count_observations = Input(shape=(1,), dtype='float32', name='count_observations')
    rs1_x = Input(shape=(1,), dtype='float32', name='rs1_x')
    rv1_x = Input(shape=(1,), dtype='float32', name='rv1_x')
    rs2_x = Input(shape=(1,), dtype='float32', name='rs2_x')
    rv2_x = Input(shape=(1,), dtype='float32', name='rv2_x')
    rs1_y = Input(shape=(1,), dtype='float32', name='rs1_y')
    rv1_y = Input(shape=(1,), dtype='float32', name='rv1_y')
    rs2_y = Input(shape=(1,), dtype='float32', name='rs2_y')
    rv2_y = Input(shape=(1,), dtype='float32', name='rv2_y')
    total_reserv_sum = Input(shape=(1,), dtype='float32', name='total_reserv_sum')
    total_reserv_mean = Input(shape=(1,), dtype='float32', name='total_reserv_mean')
    total_reserv_dt_diff_mean = Input(shape=(1,), dtype='float32', name='total_reserv_dt_diff_mean')
    date_int = Input(shape=(1,), dtype='float32', name='date_int')
    var_max_lat = Input(shape=(1,), dtype='float32', name='var_max_lat')
    var_max_long = Input(shape=(1,), dtype='float32', name='var_max_long')
    lon_plus_lat = Input(shape=(1,), dtype='float32', name='lon_plus_lat')

    date_emb = keras.layers.concatenate([dow_emb, month_emb, year, holiday_flg])
    date_emb = Dense(5, activation='sigmoid', name='date_merged_emb')(date_emb)

    cat_layer = keras.layers.concatenate([holiday_flg, min_visitors, mean_visitors,
                    median_visitors, max_visitors, count_observations, rs1_x, rv1_x,
                    rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y,
                    total_reserv_sum, total_reserv_mean, total_reserv_dt_diff_mean,
                    date_int, var_max_lat, var_max_long, lon_plus_lat,
                    date_emb, air_area_name_emb, air_genre_name_emb,
                    air_area_code_emb, air_genre_code_emb, air_store_id_emb])

    m = Dense(hidden1_neurons, name='hidden1',
             kernel_initializer=keras.initializers.RandomNormal(mean=0.0,
                            stddev=0.05, seed=None))(cat_layer)
    m = keras.layers.LeakyReLU(alpha=0.2)(m)
    m = keras.layers.BatchNormalization()(m)
    
    m1 = Dense(hidden2_neurons, name='sub1')(m)
    m1 = keras.layers.LeakyReLU(alpha=0.2)(m1)
    m = Dense(1, activation='relu')(m1)

    #27?
    inp_ten = [
        holiday_flg, min_visitors, mean_visitors, median_visitors, max_visitors, count_observations,
        rs1_x, rv1_x, rs2_x, rv2_x, rs1_y, rv1_y, rs2_y, rv2_y, total_reserv_sum, total_reserv_mean,
        total_reserv_dt_diff_mean, date_int, var_max_lat, var_max_long, lon_plus_lat,
        dow, year, month, air_store_id, air_area_code, air_genre_code
    ]
    inp_ten += air_area_name
    inp_ten += air_genre_name
    model = keras.Model(inp_ten, m)
    model.compile(loss='mse', optimizer='rmsprop', metrics=['acc'])

    return model

In [44]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3,
                    n_estimators=200, subsample=0.8, max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =10)
model4 = get_nn_complete_model(train, hidden2_neurons=12)

In [ ]:
model1.fit(train[col], np.log1p(train['visitors'].values))
print("Model1 trained")
model2.fit(train[col], np.log1p(train['visitors'].values))
print("Model2 trained")
model3.fit(train[col], np.log1p(train['visitors'].values))
print("Model3 trained")

In [45]:
for i in range(5):
    model4.fit(nn_train[0], nn_train[1], epochs=3, verbose=1,
        batch_size=256, shuffle=True, validation_split=0.15)
    model4.fit(nn_train[0], nn_train[1], epochs=8, verbose=0,
        batch_size=256, shuffle=True)
print("Model4 trained")

preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])
preds4 = pd.Series(model4.predict(nn_train[0]).reshape(-1)).clip(0, 6.8).values

print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))
print('RMSE NeuralNetwork: ', RMSLE(np.log1p(train['visitors'].values), preds4))

preds1 = model1.predict(test[col])
preds2 = model2.predict(test[col])
preds3 = model3.predict(test[col])
preds4 = pd.Series(model4.predict(nn_test[0]).reshape(-1)).clip(0, 6.8).values

test['visitors'] = 0.2*preds1+0.2*preds2+0.3*preds3+0.3*preds4
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
print("Model predictions done.")
# del train; del data;

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 39 array(s), but instead got the following list of 69 arrays: [array([ 0.,  0.,  0., ...,  0.,  0.,  0.]), array([ 0.06015038,  0.02255639,  0.03759398, ...,  0.02255639,
        0.02255639,  0.02255639]), array([ 0.16737849,  0.14345154,  0.24077886, ...,  0.04...

In [ ]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../input/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub2 = sub2.fillna(-1)

In [ ]:
def final_visitors(x, alt=False):
    visitors_x, visitors_y = x['visitors_x'], x['visitors_y']
    if x['visitors_y'] == -1:
        return visitors_x
    else:
        return 0.7*visitors_x + 0.3*visitors_y* 1.1

sub_merge = pd.merge(sub1, sub2, on='id', how='inner')
sub_merge['visitors'] = sub_merge.apply(lambda x: final_visitors(x), axis=1)
print("Done")
sub_merge[['id', 'visitors']].to_csv('submission_rs_recruit_v19_keras_fe.csv', index=False)